In [ ]:
# 1. Install PortAudio using Homebrew
%brew install portaudio

# 2. Then install PyAudio via pip
%pip install pyaudio


Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyaudio: filename=pyaudio-0.2.14-cp39-cp39-macosx_10_9_universal2.whl size=39062 sha256=6f96d2d895d126979513236975319c4addbf3347731e48badeca97d2214bb6b9
  Stored in directory: /Users/hemakoti/Library/Caches/pip/wheels/28/d3/62/6ad369dc09fe82e1c9ceb83601a800eb305b901df7789aa550
Successfully built pyaudio
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install pyttsx3 SpeechRecognition requests python-dateutil


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 36.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pyttsx3
import speech_recognition as sr
import datetime
import os
import requests
import json
from dateutil import parser
import time

# ========== CONFIG ==========
ESP32_IP = "http://192.168.1.100"
SCHEDULE_FILE = "schedule.json"
RELAY_COUNT = 8
TRIGGER_WORD = "jarvis"
# ============================

# ========== INIT ==========
engine = pyttsx3.init()
voices = engine.getProperty("voices")
engine.setProperty('voice', voices[14].id)
engine.setProperty("rate", 190)
engine.setProperty("volume", 1)
# ==========================

# ========== SPEAK ==========
def speak(text):
    print(f"Speaking: {text}")
    engine.say(text)
    engine.runAndWait()

# ========== COMMAND RECOGNITION ==========
def take_command(timeout=300):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        try:
            audio = recognizer.listen(source, timeout=timeout, phrase_time_limit=10)
            query = recognizer.recognize_google(audio).lower()
            print(f"User said: {query}")
            return query
        except:
            return "none"

# ========== UTILS ==========
def word_to_number(word):
    numbers = {
        "one": "1", "two": "2", "three": "3", "four": "4",
        "five": "5", "six": "6", "seven": "7", "eight": "8",
        "all": "all"
    }
    return numbers.get(word.lower(), None)

def parse_datetime(input_text):
    try:
        return parser.parse(input_text, fuzzy=True)
    except:
        return None

def load_schedule():
    try:
        with open(SCHEDULE_FILE, "r") as f:
            return json.load(f)
    except:
        return {}

def save_schedule(schedules):
    with open(SCHEDULE_FILE, "w") as f:
        json.dump(schedules, f)

# ========== ESP32 INTERACTION ==========
def control_relay(relay_number, action):
    try:
        state = "true" if action == "on" else "false"

        if relay_number == "all":
            for i in range(1, RELAY_COUNT + 1):
                requests.get(f"{ESP32_IP}/toggle?relay={i}&state={state}")
            speak(f"All lights turned {action.upper()}.")
        else:
            response = requests.get(f"{ESP32_IP}/toggle?relay={relay_number}&state={state}")
            if response.status_code == 200:
                speak(f"Switch {relay_number} turned {action.upper()}.")
            else:
                speak(f"Failed to control switch {relay_number}.")
    except Exception as e:
        speak("Error reaching ESP32.")
        print(f"Error: {e}")

def get_sensor_data():
    try:
        response = requests.get(f"{ESP32_IP}/sensor")
        if response.status_code == 200:
            data = response.json()
            return data.get("temperature"), data.get("humidity")
        else:
            speak("Failed to get sensor data.")
            return None, None
    except:
        speak("ESP32 not reachable.")
        return None, None

def get_relay_status():
    try:
        response = requests.get(f"{ESP32_IP}/status")
        if response.status_code == 200:
            return response.json()
        return {}
    except:
        return {}

# ========== RESPOND ==========
def respond(query):
    schedules = load_schedule()

    if "time" in query:
        now = datetime.datetime.now().strftime("%H:%M:%S")
        speak(f"The time is {now}")

    elif "date" in query:
        today = datetime.datetime.now().strftime("%A, %B %d, %Y")
        speak(f"Today is {today}")

    elif "temperature" in query or "humidity" in query:
        temp, hum = get_sensor_data()
        if temp is not None:
            if "temperature" in query:
                speak(f"Current temperature is {temp} °C")
            elif "humidity" in query:
                speak(f"Humidity level is {hum} percent")

    elif "turn on" in query or "turn off" in query:
        action = "on" if "turn on" in query else "off"
        parts = query.split("light")
        if len(parts) > 1:
            light_num = word_to_number(parts[1].strip()) or parts[1].strip()
            control_relay(light_num, action)
        else:
            speak("Please specify which light to control.")

    elif "status" in query:
        temp, hum = get_sensor_data()
        statuses = get_relay_status()
        speak("System status report:")
        if temp: speak(f"Temperature: {temp}°C")
        if hum: speak(f"Humidity: {hum}%")
        for relay, state in statuses.items():
            state_str = "ON" if state == "1" else "OFF"
            speak(f"{relay.capitalize()} is {state_str}")

    elif "schedule" in query or "meeting" in query:
        if "set" in query or "add" in query:
            speak("Event name?")
            event = take_command()
            if event == "none": return
            speak("Date and time?")
            event_time_input = take_command()
            dt = parse_datetime(event_time_input)
            if dt:
                schedules[event] = dt.strftime("%Y-%m-%d %H:%M")
                save_schedule(schedules)
                speak(f"Scheduled {event} at {dt.strftime('%A, %B %d, %Y %H:%M')}")
            else:
                speak("Invalid date and time.")
        elif "what" in query or "show" in query:
            if schedules:
                speak("Here are your upcoming events:")
                for e, t in schedules.items():
                    speak(f"{e} at {t}")
            else:
                speak("You have no scheduled events.")

    elif "goodbye" in query or "exit" in query:
        speak("Goodbye Chief, going to sleep.")
        exit()

    else:
        speak("Sorry, I didn't understand the command.")

# ========== TRIGGER LISTENER ==========


def listen_for_trigger():
    recognizer = sr.Recognizer()
    while True:
        with sr.Microphone() as source:
            print("Awaiting trigger...")
            recognizer.adjust_for_ambient_noise(source)
            try:
                audio = recognizer.listen(source, timeout=300, phrase_time_limit=10)
                query = recognizer.recognize_google(audio).lower()
                if TRIGGER_WORD in query:
                    speak("Yes Chief?")
                    last_command_time = time.time()

                    while True:
                        command = take_command()
                        current_time = time.time()

                        # If no valid command is received in 2 minutes, return to trigger listening
                        if current_time - last_command_time > 120:
                            speak("No command received. Returning to standby mode.")
                            break

                        if command == "none":
                            continue

                        last_command_time = time.time()

                        if "Goodbye" in command:
                            speak("Goodbye Chief.")
                            break
                        respond(command)

            except:
                continue


# ========== STARTUP GREET ==========
def greet_user():
    hour = datetime.datetime.now().hour
    if 0 <= hour < 12:
        speak("Good morning Chief.")
    elif 12 <= hour < 18:
        speak("Good afternoon Chief.")
    else:
        speak("Good evening Chief.")
    speak("Jarvis is online. Awaiting your command.")

# ========== MAIN ==========
def main():
    greet_user()
    listen_for_trigger()

if __name__ == "__main__":
    main()


/Users/hemakoti/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Speaking: Good afternoon Chief.
Speaking: Jarvis is online. Awaiting your command.
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger...
Awaiting trigger.

In [ ]:
import pyttsx3
import speech_recognition as sr
import datetime
import os
import requests
import json
from dateutil import parser
import time

# ========== CONFIG ==========
ESP32_IP = "http://192.168.1.100"
SCHEDULE_FILE = "schedule.json"
RELAY_COUNT = 8
TRIGGER_WORD = "jarvis"
# ============================

# ========== INIT ==========
engine = pyttsx3.init()
voices = engine.getProperty("voices")
engine.setProperty('voice', voices[14].id)
engine.setProperty("rate", 190)
engine.setProperty("volume", 1)
# ==========================

# ========== SPEAK ==========
def speak(text):
    print(f"Speaking: {text}")
    engine.say(text)
    engine.runAndWait()

# ========== COMMAND RECOGNITION ==========
def take_command(timeout=300):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        try:
            audio = recognizer.listen(source, timeout=timeout, phrase_time_limit=10)
            query = recognizer.recognize_google(audio).lower()
            print(f"User said: {query}")
            return query
        except:
            return "none"

# ========== UTILS ==========
def word_to_number(word):
    numbers = {
        "one": "1", "two": "2", "three": "3", "four": "4",
        "five": "5", "six": "6", "seven": "7", "eight": "8",
        "all": "all"
    }
    return numbers.get(word.lower(), None)

def parse_datetime(input_text):
    try:
        return parser.parse(input_text, fuzzy=True)
    except:
        return None

def load_schedule():
    try:
        with open(SCHEDULE_FILE, "r") as f:
            return json.load(f)
    except:
        return {}

def save_schedule(schedules):
    with open(SCHEDULE_FILE, "w") as f:
        json.dump(schedules, f)

# ========== ESP32 INTERACTION ==========
def control_relay(relay_number, action):
    try:
        state = "true" if action == "on" else "false"

        if relay_number == "all":
            for i in range(1, RELAY_COUNT + 1):
                requests.get(f"{ESP32_IP}/toggle?relay={i}&state={state}")
            speak(f"All lights turned {action.upper()}.")
        else:
            response = requests.get(f"{ESP32_IP}/toggle?relay={relay_number}&state={state}")
            if response.status_code == 200:
                if action == "on":
                    speak(f"focus lamp turned ON.")
                #speak(f"Switch {relay_number} turned {action.upper()}.")
            else:
                speak(f"Failed to control switch {relay_number}.")
    except Exception as e:
        speak("Error reaching ESP32.")
        print(f"Error: {e}")

def get_sensor_data():
    try:
        response = requests.get(f"{ESP32_IP}/sensor")
        if response.status_code == 200:
            data = response.json()
            return data.get("temperature"), data.get("humidity")
        else:
            speak("Failed to get sensor data.")
            return None, None
    except:
        speak("ESP32 not reachable.")
        return None, None

def get_relay_status():
    try:
        response = requests.get(f"{ESP32_IP}/status")
        if response.status_code == 200:
            return response.json()
        return {}
    except:
        return {}

# ========== RESPOND ==========
def respond(query):
    schedules = load_schedule()

    if "time" in query:
        now = datetime.datetime.now().strftime("%H:%M:%S")
        speak(f"The time is {now}")

    elif "date" in query:
        today = datetime.datetime.now().strftime("%A, %B %d, %Y")
        speak(f"Today is {today}")

    elif "temperature" in query or "humidity" in query:
        temp, hum = get_sensor_data()
        if temp is not None:
            if "temperature" in query:
                speak(f"Current temperature is {temp} °C")
            elif "humidity" in query:
                speak(f"Humidity level is {hum} percent")

    elif "turn on" in query or "turn off" in query or "focus lamp" in query:
        action = "on" if "turn on" in query else "off"

        if "focus lamp" in query:
            # Force control Relay 2 (relay number 1 in 0-indexed)
            control_relay("1", action)
        else:
            parts = query.split("light")
            if len(parts) > 1:
                light_num = word_to_number(parts[1].strip()) or parts[1].strip()
                control_relay(light_num, action)
            else:
                speak("Please specify which light to control.")

    elif "status" in query:
        temp, hum = get_sensor_data()
        statuses = get_relay_status()
        speak("System status report:")
        if temp: speak(f"Temperature: {temp}°C")
        if hum: speak(f"Humidity: {hum}%")
        for relay, state in statuses.items():
            state_str = "ON" if state == "1" else "OFF"
            speak(f"{relay.capitalize()} is {state_str}")

    elif "schedule" in query or "meeting" in query:
        if "set" in query or "add" in query:
            speak("Event name?")
            event = take_command()
            if event == "none": return
            speak("Date and time?")
            event_time_input = take_command()
            dt = parse_datetime(event_time_input)
            if dt:
                schedules[event] = dt.strftime("%Y-%m-%d %H:%M")
                save_schedule(schedules)
                speak(f"Scheduled {event} at {dt.strftime('%A, %B %d, %Y %H:%M')}")
            else:
                speak("Invalid date and time.")
        elif "what" in query or "show" in query:
            if schedules:
                speak("Here are your upcoming events:")
                for e, t in schedules.items():
                    speak(f"{e} at {t}")
            else:
                speak("You have no scheduled events.")

    elif "goodbye" in query or "exit" in query:
        speak("Goodbye Chief, going to sleep.")
        exit()

    else:
        speak("Sorry, I didn't understand the command.")

# ========== TRIGGER LISTENER ==========


def listen_for_trigger():
    recognizer = sr.Recognizer()
    while True:
        with sr.Microphone() as source:
            print("Awaiting trigger...")
            recognizer.adjust_for_ambient_noise(source)
            try:
                audio = recognizer.listen(source, timeout=300, phrase_time_limit=10)
                query = recognizer.recognize_google(audio).lower()
                if TRIGGER_WORD in query:
                    speak("Yes Chief?")
                    last_command_time = time.time()

                    while True:
                        command = take_command()
                        current_time = time.time()

                        # If no valid command is received in 2 minutes, return to trigger listening
                        if current_time - last_command_time > 120:
                            speak("No command received. Returning to standby mode.")
                            break

                        if command == "none":
                            continue

                        last_command_time = time.time()

                        if "Goodbye" in command:
                            speak("Goodbye Chief.")
                            break
                        respond(command)

            except:
                continue


# ========== STARTUP GREET ==========
def greet_user():
    hour = datetime.datetime.now().hour
    if 0 <= hour < 12:
        speak("Good morning Chief.")
    elif 12 <= hour < 18:
        speak("Good afternoon Chief.")
    else:
        speak("Good evening Chief.")
    speak("Jarvis is online. Awaiting your command.")

# ========== MAIN ==========
def main():
    greet_user()
    listen_for_trigger()

if __name__ == "__main__":
    main()


/Users/hemakoti/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Speaking: Good evening Chief.
Speaking: Jarvis is online. Awaiting your command.
Awaiting trigger...
Speaking: Yes Chief?
Listening...
User said: focus lamp
Listening...
Listening...
Listening...
User said: diesel humidity and temperature
Speaking: Current temperature is 36.9 °C
